# 简单文本分类（Simple Text Classification）

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [2]:
# 获取IMDB数据集的训练集和测试集迭代器
train_iter = IMDB(split='train')    # 下载IMDB数据集训练集
test_iter = IMDB(split='test')

In [3]:
# 查看数据集
from collections import Counter

label_counter = Counter()
for label, _ in train_iter:
    label_counter[label] += 1

print("Label distribution in training set:", label_counter)

Label distribution in training set: Counter({1: 12500, 2: 12500})


In [4]:
tokenizer = get_tokenizer('basic_english')  # 分割器
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [5]:
# 构建词汇表，并添加未知词元 (<unk>) 和填充词元 (<pad>)
vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  min_freq=5,  # 过滤掉出现次数少于5的词
                                  specials=["<unk>", "<pad>"],
                                  max_tokens=10002)
vocab.set_default_index(vocab["<unk>"])  # 设置默认的未知词元索引

print(vocab(['here', 'is', 'an', 'example']))

[132, 10, 41, 465]


In [6]:
text_pipeline = lambda x: vocab(tokenizer(x))  # 处理文本的管道
label_pipeline = lambda x: int(x) - 1  # 处理label的管道

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [8]:
class TextClassificationModel(nn.Module):  # 分类模型

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [9]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)